# Limpieza de texto
### Creación de los dataframe y limpieza de texto

In [3]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df1 = df['text'].apply(str)

In [4]:
# Limpieza de texto
# Primera etapa de la limpieza de texto
import re, string, unicodedata
'''
Se hace la eliminacion de texto entre corchetes, acentos, signos de puntuacion (excepto . y :), palabras con numeros.
Se eliminan los espacios de sobra
Se eliminan \r, \t, \v, \f, \a
'''
def limpiarTexto1(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))

    '''
    Eliminamos caracteres especiales: tabulador orizontal(\t), tabulador vertical(\v), 
    retorno de carro(\r), avance de pagina(\f), 
    caracter de retroceso: Marca el límite de una palabra(\b), 
    '''
    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt
df1 = df1.apply(str)
df1 = df1.apply(limpiarTexto1)

In [5]:
import re

def limpiarTexto(text):
    text = re.sub(r'[-,;+º*?¿!¡<>{}()// %&´""]',' ',text)
    text = re.sub('^ +',' ',text)
    text = re.sub('\n ','',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' +\n','\n',text)
    #text = re.sub('\. \n','\.\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('¨[... ]','',text)
    text = text.lower()
    text = re.sub(' +',' ',text)
    text = re.sub('^ +','',text)
    text = re.sub('n.º.','',text)
    text = re.sub('[a-zA-z-.]@[a-zA-Z].com','',text)
    text = re.sub('\d+','',text)
    text = re.sub(r'^ ix|iv|vi+|i+x |(i{2,3})','', text) #dejar los numeros romanos

    text = re.sub(' +\n+','\n',text)
    text = re.sub('\n+','\n',text)
    text = re.sub('\d+\n','',text)
    #text = text.strip().replace('\n','')
    return text

dfLimpio = df1.apply(limpiarTexto)

In [6]:
# Segunda fase de limpieza
# Se eliminan todos los elementos que meten ruido al texto y que no fueron eliminados en la fase de limpieza 1.
import re

def limpiarTexto2(text):
    text = re.sub('^ ',' ',text)
    text = re.sub('\n +\n','\n',text)
    text = re.sub(' +\n\n','\n',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' \n','\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('\u200b\n','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('\d+-\d+','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('[nN]º|[nN][. ]º','',text)
    text = re.sub('[º<>/]','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('[a-zA-z-.]+@[a-zA-Z]+.com','',text)
    return text

df1 = df1.apply(limpiarTexto2)

In [7]:
# Tercera fase de limpieza
# Eliminamos las lineas que no son de utilidad para el analisis o que van a afectar los resultados del mismo.
# Ejemplo de linea eliminada: las lineas que comienzan con "Se encuentra contestada en....."
import re

def limpiarTexto3(text):
    text = re.sub('[a-z1-9.]+[).-] [s|S]e encuentra contestad[a|o] .+[. \n]','',text)
    text = re.sub('[0-9]+[. ]+[yY]a fue contestado.+[.\n]','',text)
    text = re.sub('[fF]oja [1-9].+\n', '', text)
    text = re.sub('[pP]regunta[ 0-9]+[)].+\n|[rR]espuesta[ 0-9]+[)].+\n','',text)#elimina oraciones comenzadas en preguta/respuesta.
    text = re.sub('V[. ]+S.', '', text)
    #text = re.sub('[IV]+.[A-Z]{1,3}[\n.]', '', text)
    text = re.sub('[I][.][P][.]','',text)
    text = re.sub('[I][.][T][.]','',text)
    text = re.sub('[I][.][A][.]','',text)
    text = re.sub('[I][.][L][.]','',text)
    text = re.sub('[I][.][B][.]','',text)
    text = re.sub('[I][.][N][.]','',text)
    text = re.sub('[I][.][V][.]','',text)
    text = re.sub('[V][.][M][.]','',text)
    text = re.sub('[V][.][A][.]','',text)

    return text

dfLimpio = df1.apply(limpiarTexto3)
dfLimpio = dfLimpio.apply(limpiarTexto2)
#pprint(dfLimpio[0])

# Busqueda de titulos

In [27]:
# Busca títulos en mayusculas
def buscarTitulosMayusculas(text):
    # Expresiín regular para encontrar títulos en mayusculas.
    tituloMayusculas =re.compile(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.)]+[A-Z -]+[\:\.\n]') #[1-9]|
    titulosMayusculasEncontrados = []

    for m in tituloMayusculas.finditer(text):
        titulosMayusculasEncontrados.append(m.group())

    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los títulos en mayusculas
titulosMayusculas=[]
for expediente in dfLimpio:
    titulosMayusculas.append(buscarTitulosMayusculas(expediente))

dfTitulosMayusculasConStops= pd.DataFrame(titulosMayusculas)

pprint(titulosMayusculas[1])

['II). PREAMBULO.',
 'III). ANTECEDENTES DE INTERES MEDICO PRESENTE EN AUTOS.',
 'IV) EXAMEN DEL PERITO DE OFICIO.',
 'V) CONSIDERACIONES MEDICAS:',
 'VII) CONTESTACION DE PUNTOS PERICIALES.',
 'VIII) BIBLIOGRAFIA.',
 'VII) CONCLUSIONES:',
 'IX). PETITORIO.']


In [26]:
pprint(titulosMayusculas[0])

['I-PROEMIO:',
 'II-ANTECEDENTES DE INTERES MEDICO-LEGAL:',
 'III-EXAMEN FISICO DEL ACTOR:',
 'IV-CONSIDERACIONES MEDICO-LEGALES:',
 'VII-CONCLUSIONES.',
 'VII-INCAPACIDAD:',
 'IX-PETITORIO:']


In [9]:

# Ultima limpieza de titulos
# def limpiarTitulosHTML(text):
#     text = re.sub('\n','',text)
#     text = re.sub(' +',' ',text)
#     #text = re.sub('[n]','',text)
#     #text = re.sub('[\\\]',' ',text)
#     return text
# titulosMayusculas = limpiarTitulosHTML(str(titulosMayusculas))
# dfTitulosMayusculasConStops = dfTitulosMayusculasConStops.apply(str)
# dfTitulosMayusculasConStops = dfTitulosMayusculasConStops.apply(limpiarTitulosHTML)


## Titulos y ubicacion

In [21]:
# Encontrar la ubicacion del titulo en el documento
titulosPosicion=[]

for titulo in titulosMayusculas[1]:
    inicioTitulo = dfLimpio[1].index(titulo)
    palabrasPorTitulo = len(titulo)
    finalTitulo = inicioTitulo + palabrasPorTitulo
    titulosPosicion.append((titulo, inicioTitulo, finalTitulo, palabrasPorTitulo))

pprint(titulosPosicion)


#anotar parrafos entre titulos.
#parrafo = texto[inicio:inicio+1]
#mostrar las secciones
#estadisiticas de titulos interdocumentos

[('IV) EXAMEN DEL PERITO DE OFICIO.', 5035, 5067, 32),
 ('V) CONSIDERACIONES MEDICAS:', 32819, 32846, 27),
 ('VII) CONTESTACION DE PUNTOS PERICIALES.', 43724, 43763, 39),
 ('VIII) BIBLIOGRAFIA.', 45065, 45084, 19),
 ('VII) CONCLUSIONES:', 45833, 45851, 18)]


In [ ]:
#pprint(dfLimpio[1][32819:43723])
parrafoConsideraciones = dfLimpio[1][32819:43723]
pprint(parrafoConsideraciones)

# PRUEBAS

In [19]:
# Creamos un dataframe para colocar el salto de linea en formato HTML
dfLimpioHTML = dfLimpio.apply(str)
def limpiarTextoHTML(text):
    text = text.strip().replace('\n','<br>')
    return text

dfLimpioHTML = dfLimpioHTML.apply(limpiarTextoHTML)
parrafoConsideraciones = limpiarTextoHTML(parrafoConsideraciones)

In [28]:
colors = {
    'procedimientos': '#cd5c5c',
    'cuerpos': '#99ccff',
    'afecciones': '#ffa500'
}

body = f'''
        <p>
        {dfLimpioHTML[1]}
        </p>
        <br>
        '''
body = body.replace(parrafoConsideraciones,f'<span style="background-color: #99ccff">{parrafoConsideraciones}</span>')
for titulo in titulosMayusculas[1]:
    body = body.replace(titulo,f'<span style="background-color: #cd5c5c">{titulo}</span>')
with open('parrafos.html','w', encoding="utf-8") as f:
    f.write(body)